In [1]:
import json
from datetime import datetime
from dateutil import tz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn


In [2]:
with open('../newest_ts', 'r') as f:
    ts = f.read()

data_path = f"../data/data_{ts}.json"
with open(data_path, 'r') as f:
    data = json.load(f)

In [3]:
def ts2dt(ts, format='%Y-%m-%d %H:%M:%S'):
    return (datetime.utcfromtimestamp(int(ts))
                    .replace(tzinfo=tz.tzutc())
                    .astimezone(tz.tzlocal())
                    .strftime(format))

In [4]:
data.keys()

dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'current', 'hourly', 'daily'])

In [5]:
data['current']

{'dt': 1629628449,
 'sunrise': 1629583360,
 'sunset': 1629629370,
 'temp': 31.05,
 'feels_like': 38.05,
 'pressure': 1006,
 'humidity': 76,
 'dew_point': 26.32,
 'uvi': 0,
 'clouds': 30,
 'visibility': 10000,
 'wind_speed': 3.64,
 'wind_deg': 199,
 'wind_gust': 5.05,
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}]}

In [6]:
[ts2dt(hrdt['dt'], format='%m-%d %H:%M') for hrdt in data['hourly']]

['08-22 18:00',
 '08-22 19:00',
 '08-22 20:00',
 '08-22 21:00',
 '08-22 22:00',
 '08-22 23:00',
 '08-23 00:00',
 '08-23 01:00',
 '08-23 02:00',
 '08-23 03:00',
 '08-23 04:00',
 '08-23 05:00',
 '08-23 06:00',
 '08-23 07:00',
 '08-23 08:00',
 '08-23 09:00',
 '08-23 10:00',
 '08-23 11:00',
 '08-23 12:00',
 '08-23 13:00',
 '08-23 14:00',
 '08-23 15:00',
 '08-23 16:00',
 '08-23 17:00',
 '08-23 18:00',
 '08-23 19:00',
 '08-23 20:00',
 '08-23 21:00',
 '08-23 22:00',
 '08-23 23:00',
 '08-24 00:00',
 '08-24 01:00',
 '08-24 02:00',
 '08-24 03:00',
 '08-24 04:00',
 '08-24 05:00',
 '08-24 06:00',
 '08-24 07:00',
 '08-24 08:00',
 '08-24 09:00',
 '08-24 10:00',
 '08-24 11:00',
 '08-24 12:00',
 '08-24 13:00',
 '08-24 14:00',
 '08-24 15:00',
 '08-24 16:00',
 '08-24 17:00']

In [7]:
data['hourly'][0]

{'dt': 1629626400,
 'temp': 30.73,
 'feels_like': 37.73,
 'pressure': 1006,
 'humidity': 75,
 'dew_point': 25.78,
 'uvi': 0.25,
 'clouds': 31,
 'visibility': 10000,
 'wind_speed': 4.16,
 'wind_deg': 204,
 'wind_gust': 5.09,
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'pop': 0}

In [8]:
df_hourly = pd.DataFrame(data['hourly'])
df_daily = pd.DataFrame(data['daily'])

In [9]:
df_hourly.head()

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather,pop,rain
0,1629626400,30.73,37.73,1006,75,25.78,0.25,31,10000,4.16,204,5.09,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.00,NaN
1,1629630000,31.05,38.05,1006,76,26.32,0.00,30,10000,3.64,199,5.05,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.00,NaN
2,1629633600,30.53,37.53,1006,77,26.04,0.00,29,10000,3.50,197,4.87,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.00,NaN
3,1629637200,29.97,36.97,1006,79,25.93,0.00,22,10000,3.39,191,4.88,"[{'id': 801, 'main': 'Clouds', 'description': ...",0.08,NaN
4,1629640800,29.37,36.03,1007,81,25.77,0.00,22,10000,3.55,198,5.09,"[{'id': 801, 'main': 'Clouds', 'description': ...",0.04,NaN
